# Training Neural Networks to Infer the Peclet Number

This notebook trains convolutional neural networks to infer the D/K
ratio of 2D streampower-diffusion landscape evolution models

In [ ]:
import torch
from train_peclet_model import PecletModelTrainer
from ThreeLayerCNNRegressor import ThreeLayerCNNRegressor, JumboThreeLayerCNNRegressor
import json

from config import MODEL_STATS_PATH, DB_PATH, MODEL_DEM_PATH, MODEL_SLOPE_PATH, MODEL_ACC_PATH, MODEL_CURV_PATH, LABEL_QUERY, WEIGHTS_PATH, NN_SEEDS, NUM_EPOCHS, LEARNING_RATE, RETRAIN_MODELS


In [ ]:
with open(MODEL_STATS_PATH, 'r') as f:
    statistics = json.load(f)
retrain = False
data_types = [{'type': 'dem',
               'data_path': MODEL_DEM_PATH},
              {'type': 'accumulation',
               'data_path': MODEL_ACC_PATH},
              {'type': 'slope',
               'data_path': MODEL_SLOPE_PATH},
              {'type': 'curvature',
               'data_path': MODEL_CURV_PATH}
               ]


In [ ]:
for seed in NN_SEEDS:
    for data in data_types:
        torch.manual_seed(seed)
        weights_path = WEIGHTS_PATH / f"{data['type']}_{seed}_weights.pt"
        if not os.path.exists(weights_path) or RETRAIN_MODELS:
            trainer = PecletModelTrainer(DB_PATH,
                                        data['data_path'],
                                        ThreeLayerCNNRegressor(),
                                        LABEL_QUERY,
                                        epochs = NUM_EPOCHS,
                                        learning_rate = LEARNING_RATE,
                                        **statistics[data['type']],
                                        **statistics['labels'])
            trainer.train()
            trainer.save_weights(weights_path)
        if data['type']=='dem':
            for second_data in data_types:
                weights_path = WEIGHTS_PATH / f"dem_{second_data['type']}_{seed}_weights.pt"
                if not weights_path.exists() or RETRAIN_MODELS:
                    inputs_mean = np.stack([v['inputs_mean'] for k,v in statistics.items() if k in ('dem', second_data['type'])])[:, np.newaxis, np.newaxis]
                    inputs_std= np.stack([v['inputs_std'] for k,v in statistics.items() if k in ('dem', second_data['type'])])[:, np.newaxis, np.newaxis]
                    trainer = PecletModelTrainer(DB_PATH,
                                                 [data['data_path'], second_data['data_path']],
                                                 ThreeLayerCNNRegressor(channels=2),
                                                 LABEL_QUERY,
                                                 epochs = NUM_EPOCHS,
                                                 learning_rate = LEARNING_RATE,
                                                 inputs_mean = inputs_mean,
                                                 inputs_std = inputs_std,
                                                 **statistics['labels']
                                                 )


We also train a "jumbo" model with larger layers.

In [ ]:
weights_path = os.path.join(WEIGHTS_dir, 'dem_jumbo_run_weights.pt')
if not os.path.exists(weights_path) or RETRAIN_MODELS:
    torch.manual_seed(NN_SEEDS[0])
    trainer = PecletModelTrainer(DB_PATH,
                                MODEL_DEM_PATH,
                                JumboThreeLayerCNNRegressor,
                                LABEL_QUERY,
                                epochs = NUM_EPOCHS,
                                learning_rate = LEARNING_RATE,
                                **statistics['dem'],
                                **statistics['labels'])
    trainer.train()
    trainer.save_weights(weights_path)


\*